In [6]:
%pylab inline
import rlcompleter, readline
readline.parse_and_bind('tab: complete')
import os
import csv
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from pyspark import SparkContext
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
from pyspark.sql.functions import monotonicallyIncreasingId

Populating the interactive namespace from numpy and matplotlib


In [45]:
data = sc.textFile('../dac/train.txt', 10000)
sample = sc.textFile('../dac_sample/dac_sample.txt').map(lambda x: x.split('\t'))
sample = sample.map(lambda x: x[1:])
split = data.map(lambda x: x.split('\t'))
label = split.map(lambda x: x[0])
entire_train = split.map(lambda x: x[1:])

In [46]:
train10M, train38M = entire_train.randomSplit([10,38])
train5M, validation2M, test3M = train10M.randomSplit([5,2,3])

In [276]:
df_train = sqlContext.createDataFrame(train5M, ['C'+str(i) for i in range(39)])
df_sample = sqlContext.createDataFrame(sample, ['C'+str(i) for i in range(39)])

#Convert String Column to Numeric

In [277]:
def changeDataType(df):
    new = df    
    for i in range(13):
        new = new.withColumn('C'+str(i), new['C'+str(i)].cast('int'))
    return new

In [284]:
new_sample = changeDataType(df_sample)
new_sample

DataFrame[C0: int, C1: int, C2: int, C3: int, C4: int, C5: int, C6: int, C7: int, C8: int, C9: int, C10: int, C11: int, C12: int, C13: string, C14: string, C15: string, C16: string, C17: string, C18: string, C19: string, C20: string, C21: string, C22: string, C23: string, C24: string, C25: string, C26: string, C27: string, C28: string, C29: string, C30: string, C31: string, C32: string, C33: string, C34: string, C35: string, C36: string, C37: string, C38: string]

#Statistics

In [279]:
def caculateMean(df):
    means = []
    for i in range(13):
        means.append(new_sample.groupBy().mean('C'+str(i)).collect()[0][0])
    return means

In [285]:
means = caculateMean(new_sample)
new_sample

DataFrame[C0: int, C1: int, C2: int, C3: int, C4: int, C5: int, C6: int, C7: int, C8: int, C9: int, C10: int, C11: int, C12: int, C13: string, C14: string, C15: string, C16: string, C17: string, C18: string, C19: string, C20: string, C21: string, C22: string, C23: string, C24: string, C25: string, C26: string, C27: string, C28: string, C29: string, C30: string, C31: string, C32: string, C33: string, C34: string, C35: string, C36: string, C37: string, C38: string]

In [281]:
def calculateStd(df):
    stds = []
    for i in range(13):
        std = float(new_sample.describe('C'+str(i)).collect()[2][1])
        stds.append(std)
    return stds

In [288]:
stds = calculateStd(new_sample)
new_sample

DataFrame[C0: int, C1: int, C2: int, C3: int, C4: int, C5: int, C6: int, C7: int, C8: int, C9: int, C10: int, C11: int, C12: int, C13: string, C14: string, C15: string, C16: string, C17: string, C18: string, C19: string, C20: string, C21: string, C22: string, C23: string, C24: string, C25: string, C26: string, C27: string, C28: string, C29: string, C30: string, C31: string, C32: string, C33: string, C34: string, C35: string, C36: string, C37: string, C38: string]

#Missing Value
1) fill in median for numeric data
2) fill in 'NA' for categoric data

In [353]:
def fillMissing(df):
    cols_name = ['C'+str(i) for i in range(39)]
    fill_data = means + ["NA"]*26
    dic = dict(zip(cols_name,fill_data))
    new = df.fillna(dic)
    return new

In [356]:
filled = fillMissing(new_sample)

#Label Encoding

In [348]:
means[0], means[2]

(3.7687229028369944, 40.74491334767238)

In [343]:
new_sample.show()

+----+---+----+----+-----+----+---+---+----+----+---+----+----+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|  C0| C1|  C2|  C3|   C4|  C5| C6| C7|  C8|  C9|C10| C11| C12|     C13|     C14|     C15|     C16|     C17|     C18|     C19|     C20|     C21|     C22|     C23|     C24|     C25|     C26|     C27|     C28|     C29|     C30|     C31|     C32|     C33|     C34|     C35|     C36|     C37|     C38|
+----+---+----+----+-----+----+---+---+----+----+---+----+----+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|   1|  1|   5|   0| 1382|   4| 15|  2| 181|   1|  2|null|   2|68fd1e64|80e26c9b|fb936136|7b4723c4|25c83c9